In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import cv2
import ffmpeg
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [2]:
local = os.getcwd()
path_aei = local + '/aei/aei*'
path_Carti = local + '/Output_file/Outin*'
path_Count = local + '/aeCount/aeCount*'
filenames_aei = sorted(glob.glob(path_aei))
filenames_Carti = sorted(glob.glob(path_Carti))
filenames_Count = sorted(glob.glob(path_Count))
len(filenames_aei), len(filenames_Carti), len(filenames_Count)

(0, 0, 0)

In [3]:
batch_no = 3
figures_aei_output = filenames_aei[::batch_no]
figures_Carti_output = filenames_Carti[::batch_no]
figures_Count_output = filenames_Count[::batch_no]

len(figures_aei_output), len(figures_Carti_output), len(figures_Count_output)

(0, 0, 0)

In [4]:
# for step, figures_Count in enumerate(figures_Count_output):
#     data = np.loadtxt(figures_Count)
#     ae1 = data[0:250]
#     plt.figure(figsize=(8, 6), dpi=80)
#     ax = sns.heatmap(ae1)
#     # ax.set_xlim(0,120)
#     # ax.set_ylim(0,56)
#     ax.invert_yaxis()
#     plt.show()

In [5]:
df = pd.read_table(figures_aei_output[0], sep = ('\s+'), names = ['time', 'i', 'a', 'e', 'inc', 'Omega', 'w', 'Theta', 'E', 'M', 'm', 'r'])
sort = df.sort_values(by = ['a']).reset_index(drop=True)



IndexError: list index out of range

In [ ]:
def plot_aei_object(ax, object_index, size, color, data):
    object_data = data.loc[data.i == object_index]
    if len(object_data.index) != 0:
        e, a = float(object_data.e), float(object_data.a)
        a_x = np.linspace(0.3, 6., 100)
        aphelion, perihelion = a * (1 + e), a * (1 - e)
        eMassQ, eMassq = 1. - (aphelion / a_x), (perihelion / a_x) - 1.
        region_data = pd.DataFrame(data = {'a': a_x, 'e': eMassQ, 'e_': eMassq})
        ax = sns.lineplot(data = region_data, x = 'a', y = 'e', color = 'red', alpha = 0.3, zorder=10)
        ax = sns.lineplot(data = region_data, x = 'a', y = 'e_', color = 'red', alpha = 0.3, zorder=10)
        ax.scatter(object_data.a, object_data.e, s = size, c = color, zorder = 100)
        
def plot_aei_scatter(ax, sort, data):
    temp = data.copy()
    temp['i'] = temp['i'].replace(dict(zip(sort.i, sort.index)))
    return sns.scatterplot(data = temp, x = 'a', y = 'e', hue = 'i', size = 'm', palette = 'viridis', legend = False, zorder=15)
    
def plot_Carti_object(ax, object_index, size, color, data):
    object_data = data.loc[data.i == object_index]
    if len(object_data.index) != 0:
        ax.scatter(object_data.x, object_data.y, s = size, c = color, zorder = 1000)
        
def plot_Carti_scatter(ax, sort, data):
    temp = data.copy()
    temp['i'] = temp['i'].replace(dict(zip(sort.i, sort.index)))
    ax.scatter(temp.x, temp.y, s = 10e5 * temp.r, c = temp.i, cmap = 'viridis')

In [ ]:
for step, (figure_aei, figure_Carti) in enumerate(zip(figures_aei_output, figures_Carti_output)):
    #extract dataset from file
    plot_aei = pd.read_table(figure_aei, sep = ('\s+'), names = ['time', 'i', 'a', 'e', 'inc', 'Omega', 'w', 'Theta', 'E', 'M', 'm', 'r'])
    plot_Carti = pd.read_table(figure_Carti, sep = ('\s+'), names = ['time', 'i', 'm', 'r', 'x', 'y', 'z', 'vx', 'vy', 'vz', 'Sx', 'Sy', 'Sz', 'amin', 'amax', 'emin', 'emax', 'aecount', 'aecountT', 'enccountT', 'test'])
    
    #locate the heavyiest objects for perhilion and aphelion line plot
    max_mass = plot_aei.loc[plot_aei.m == max(plot_aei.m)]
    # print(max_mass)
    
    #title and time series
    title = str(step).zfill(6)
    time = str(int(plot_aei.time[0] / 1000000.))
    
    #plotting config
    fig = plt.figure(figsize = (20, 10), dpi = 80)
    fig.suptitle(title + ' / ' + time + ' Myr', fontsize = 16)
    
    #plots
    #Left plot
    ax = fig.add_subplot(121)
    ax.set_xlim(0, frame_size)
    ax.set_ylim(0, 1.)
    ax.set_zorder(1)
    ax.set_frame_on(False)
    ax.grid(True)
    ax = plot_aei_scatter(ax, sort, plot_aei)
    
    ax2 = ax.twinx()
    ax2.axvspan(0, frame_size, facecolor="#e6e6e6")
    ax2 = sns.histplot(data = plot_aei, x = 'a', alpha = 0.1, weights = 'm', element = 'step', bins = 100)
    ax2.set_yticks(np.arange(0, 5e-6, 1.e-6))
    ax2.set_ylim(0, 5.0e-6)
    ax2.set_zorder(0)
        
    #Right plot
    ax4 = fig.add_subplot(122)
    ax4.set_xlim(-frame_size, frame_size)
    ax4.set_ylim(-frame_size, frame_size)
    plot_Carti_scatter(ax4, sort, plot_Carti)
    ax4.scatter(0, 0, s = 100, c = 'tab:red')
    ax4.grid(False)
    ax4.set_facecolor('gray')
    fig_name = f'/figs/{title}.png'
    
    plt.show()
    # plt.savefig(local+fig_name, dpi=300)
    plt.close()

In [ ]:
image_folder = 'figs'
video_name = 'ae_giants.mp4'

images = [img for img in sorted(os.listdir(image_folder)) if img.endswith(".png")]
frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video = cv2.VideoWriter(video_name, fourcc, 20, (width,height))

for image in images:
    video.write(cv2.imread(os.path.join(image_folder, image)))

cv2.destroyAllWindows()
video.release()
        